In [0]:
## FUNÇÕES UTILIZADAS ==> (when + otherwise) / date_format(current_timestap()) / (rpad) / (cast) / (substring)

In [0]:
# IMPORTANDO BIBLIOTECAS 

import pyspark.sql.functions
from pyspark.sql.types import StructType, StructField , StringType , IntegerType , FloatType
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

# CRIANDO SESSÃO DO SPARK 

spark = (SparkSession.builder
        .master("local(*)")
        .appName('pyspark-7')
        .getOrCreate()
        )

In [0]:
dados = (

    ['1','Celta','Chevrolet',2004,18000,"Não"],
    ['2','Gol','volkswagen',2020,46320,"Sim"],
    ['3','Corsa Classic', 'Chevrolet',2008,15000,"Sim"],
    ['4','Palio','Fiat',1998,8900,"Não"],
    ['5','ASX','Mitsubishi',2016,65000,"Sim"],
    ['6','Corolla','Toyota',2018,89990,"Sim"],
    ['7','Civic','Honda',2019,89990,"Sim"],
    ['8','Eco Sport','Ford',2017,69299,"Sim"],
    ['9','Onix','Chevrolet',2020,58990,"Sim"]

)

#SCHEMA

colunas = (StructType([
    StructField('cod' , StringType(), True),
    StructField('Carro' , StringType(), True),
    StructField('Marca' , StringType(), True),
    StructField('Ano' , IntegerType(), True),
    StructField('Preco' , IntegerType(), True),
    StructField('Flex' , StringType(), True)
]))

# CRIANDO DATA FRAME FINAL 

df = spark.createDataFrame(data = dados , schema = colunas)

df.display()


cod,Carro,Marca,Ano,Preco,Flex
1,Celta,Chevrolet,2004,18000,Não
2,Gol,volkswagen,2020,46320,Sim
3,Corsa Classic,Chevrolet,2008,15000,Sim
4,Palio,Fiat,1998,8900,Não
5,ASX,Mitsubishi,2016,65000,Sim
6,Corolla,Toyota,2018,89990,Sim
7,Civic,Honda,2019,89990,Sim
8,Eco Sport,Ford,2017,69299,Sim
9,Onix,Chevrolet,2020,58990,Sim


In [0]:
### ==>  UTILIZANDO CALCULOS DIRETAMENTE NAS COLUNAS 

# 1) CRIE UMA COLUNA (PRAZO) E ADD + 100,00 AO PREÇO ORIGINAL 

df = df.withColumn('A Prazo' , col('Preco') + 100)

df.display()

cod,Carro,Marca,Ano,Preco,Flex,A Prazo
1,Celta,Chevrolet,2004,18000,Não,18100
2,Gol,volkswagen,2020,46320,Sim,46420
3,Corsa Classic,Chevrolet,2008,15000,Sim,15100
4,Palio,Fiat,1998,8900,Não,9000
5,ASX,Mitsubishi,2016,65000,Sim,65100
6,Corolla,Toyota,2018,89990,Sim,90090
7,Civic,Honda,2019,89990,Sim,90090
8,Eco Sport,Ford,2017,69299,Sim,69399
9,Onix,Chevrolet,2020,58990,Sim,59090


In [0]:
# 2) CRIE UMA COLUNA (Á VISTA) E DÊ UM DESCONTO DE (10 %) DO (PREÇO ORIGINAL)


#df = df.withColumn('Preco', col('Preco').cast('double'))


df = df.withColumn('A vista' , col('Preco') - (col('Preco') * 0.10) )

df.show()

+---+-------------+----------+----+-----+----+-------+-------+
|cod|        Carro|     Marca| Ano|Preco|Flex|A Prazo|A vista|
+---+-------------+----------+----+-----+----+-------+-------+
|  1|        Celta| Chevrolet|2004|18000| Não|  18100|16200.0|
|  2|          Gol|volkswagen|2020|46320| Sim|  46420|41688.0|
|  3|Corsa Classic| Chevrolet|2008|15000| Sim|  15100|13500.0|
|  4|        Palio|      Fiat|1998| 8900| Não|   9000| 8010.0|
|  5|          ASX|Mitsubishi|2016|65000| Sim|  65100|58500.0|
|  6|      Corolla|    Toyota|2018|89990| Sim|  90090|80991.0|
|  7|        Civic|     Honda|2019|89990| Sim|  90090|80991.0|
|  8|    Eco Sport|      Ford|2017|69299| Sim|  69399|62369.1|
|  9|         Onix| Chevrolet|2020|58990| Sim|  59090|53091.0|
+---+-------------+----------+----+-----+----+-------+-------+



In [0]:
# 3) FAÇA UMA COLUNA (ABS) PARA OS CARROS QUE POSSUEM (ANO) > (2012)

df = df.withColumn('ABS' , when(col('Ano') > 2012 , lit('sim')).otherwise('não'))

df.show()

+---+-------------+----------+----+-----+----+-------+-------+---+
|cod|        Carro|     Marca| Ano|Preco|Flex|A Prazo|A vista|ABS|
+---+-------------+----------+----+-----+----+-------+-------+---+
|  1|        Celta| Chevrolet|2004|18000| Não|  18100|16200.0|não|
|  2|          Gol|volkswagen|2020|46320| Sim|  46420|41688.0|sim|
|  3|Corsa Classic| Chevrolet|2008|15000| Sim|  15100|13500.0|não|
|  4|        Palio|      Fiat|1998| 8900| Não|   9000| 8010.0|não|
|  5|          ASX|Mitsubishi|2016|65000| Sim|  65100|58500.0|sim|
|  6|      Corolla|    Toyota|2018|89990| Sim|  90090|80991.0|sim|
|  7|        Civic|     Honda|2019|89990| Sim|  90090|80991.0|sim|
|  8|    Eco Sport|      Ford|2017|69299| Sim|  69399|62369.1|sim|
|  9|         Onix| Chevrolet|2020|58990| Sim|  59090|53091.0|sim|
+---+-------------+----------+----+-----+----+-------+-------+---+



In [0]:
# 4) CRIE UMA COLUNA (USO) E DEFINA QUANTOS ANOS DE USO TEM O VEICULO 

# df = df.withColumn('anos' , date_format(current_timestamp(), 'yyyy').cast('integer') - substring((df.data_nasc), 7,4).cast('integer'))


df = df.withColumn('Uso' , date_format(current_timestamp(), 'yyyy' ).cast('int') - substring((df.Ano),1,4 ).cast('int'))

df.show()

+---+-------------+----------+----+-----+----+-------+-------+---+---+
|cod|        Carro|     Marca| Ano|Preco|Flex|A Prazo|A vista|ABS|Uso|
+---+-------------+----------+----+-----+----+-------+-------+---+---+
|  1|        Celta| Chevrolet|2004|18000| Não|  18100|16200.0|não| 18|
|  2|          Gol|volkswagen|2020|46320| Sim|  46420|41688.0|sim|  2|
|  3|Corsa Classic| Chevrolet|2008|15000| Sim|  15100|13500.0|não| 14|
|  4|        Palio|      Fiat|1998| 8900| Não|   9000| 8010.0|não| 24|
|  5|          ASX|Mitsubishi|2016|65000| Sim|  65100|58500.0|sim|  6|
|  6|      Corolla|    Toyota|2018|89990| Sim|  90090|80991.0|sim|  4|
|  7|        Civic|     Honda|2019|89990| Sim|  90090|80991.0|sim|  3|
|  8|    Eco Sport|      Ford|2017|69299| Sim|  69399|62369.1|sim|  5|
|  9|         Onix| Chevrolet|2020|58990| Sim|  59090|53091.0|sim|  2|
+---+-------------+----------+----+-----+----+-------+-------+---+---+



In [0]:
# 5 ) ADD A PALAVRA (ANOS) AO LADO DIREITO DOS VALORES DA COLUNA (USO)

df = df.withColumn('Uso' , rpad(df.Uso , 7 , ' Anos'))

df.show()

+---+-------------+----------+----+-----+----+-------+-------+---+-------+
|cod|        Carro|     Marca| Ano|Preco|Flex|A Prazo|A vista|ABS|    Uso|
+---+-------------+----------+----+-----+----+-------+-------+---+-------+
|  1|        Celta| Chevrolet|2004|18000| Não|  18100|16200.0|não|18 Anos|
|  2|          Gol|volkswagen|2020|46320| Sim|  46420|41688.0|sim|2 Anos |
|  3|Corsa Classic| Chevrolet|2008|15000| Sim|  15100|13500.0|não|14 Anos|
|  4|        Palio|      Fiat|1998| 8900| Não|   9000| 8010.0|não|24 Anos|
|  5|          ASX|Mitsubishi|2016|65000| Sim|  65100|58500.0|sim|6 Anos |
|  6|      Corolla|    Toyota|2018|89990| Sim|  90090|80991.0|sim|4 Anos |
|  7|        Civic|     Honda|2019|89990| Sim|  90090|80991.0|sim|3 Anos |
|  8|    Eco Sport|      Ford|2017|69299| Sim|  69399|62369.1|sim|5 Anos |
|  9|         Onix| Chevrolet|2020|58990| Sim|  59090|53091.0|sim|2 Anos |
+---+-------------+----------+----+-----+----+-------+-------+---+-------+

